In [3]:


import numpy as np
from random import seed
from collections import defaultdict
from scipy.optimize import minimize
SEED = 1122334455
seed(SEED) # set the random seed so that the random permutations can be reproduced again
np.random.seed(SEED)

def print_classifier_fairness_stats(acc_arr, correlation_dict_arr, cov_dict_arr, s_attr_name):
    correlation_dict = get_avg_correlation_dict(correlation_dict_arr)
    #print(correlation_dict)
    try:
        non_prot_pos =correlation_dict[s_attr_name][1][1]
    except:
        non_prot_pos = 0
    try:
        prot_pos = correlation_dict[s_attr_name][0][1]
    except:
        prot_pos = 0

    print ("Accuracy: %0.2f" % (np.mean(acc_arr)))
    print ("Protected/non-protected in +ve class: %0.0f%% / %0.0f%%" % (prot_pos, non_prot_pos))
    print ("Covariance between sensitive feature and decision from distance boundary : %0.3f" % (np.mean([v[s_attr_name] for v in cov_dict_arr])))

def get_one_hot_encoding(in_arr):
    """
        input: 1-D arr with int vals -- if not int vals, will raise an error
        output: m (ndarray): one-hot encoded matrix
                d (dict): also returns a dictionary original_val -> column in encoded matrix
    """

    for k in in_arr:
        if str(type(k)) != "<type 'numpy.float64'>" and type(k) != int and type(k) != np.int64:
            print (str(type(k)))
            print ("************* ERROR: Input arr does not have integer types")
            return None
        
    in_arr = np.array(in_arr, dtype=int)
    assert(len(in_arr.shape)==1) # no column, means it was a 1-D arr
    attr_vals_uniq_sorted = sorted(list(set(in_arr)))
    num_uniq_vals = len(attr_vals_uniq_sorted)
    if (num_uniq_vals == 2) and (attr_vals_uniq_sorted[0] == 0 and attr_vals_uniq_sorted[1] == 1):
        return in_arr, None

    
    index_dict = {} # value to the column number
    for i in range(0,len(attr_vals_uniq_sorted)):
        val = attr_vals_uniq_sorted[i]
        index_dict[val] = i

    out_arr = []    
    for i in range(0,len(in_arr)):
        tup = np.zeros(num_uniq_vals)
        val = in_arr[i]
        ind = index_dict[val]
        tup[ind] = 1 # set that value of tuple to 1
        out_arr.append(tup)

    return np.array(out_arr), index_dict

def check_accuracy(model, x_train, y_train, x_test, y_test, y_train_predicted, y_test_predicted):

    print("weight dims", len(model))
    #if not model:
    #    print ("Invalid Model ")
    #    assert(0)
    """
    returns the train/test accuracy of the model
    we either pass the model (w)
    else we pass y_predicted
    """
    if model is not None and y_test_predicted is not None:
        print ("Either the model (w) or the predicted labels should be None")
        raise Exception("Either the model (w) or the predicted labels should be None")

    if model is not None:
        print ("Model Predictions")
        y_test_predicted = np.sign(np.dot(x_test, model))
        y_train_predicted = np.sign(np.dot(x_train, model))

    def get_accuracy(y, Y_predicted):
        correct_answers = (Y_predicted == y).astype(int) # will have 1 when the prediction and the actual label match
        #print(" correct : ", correct_answers, " len : ", len(correct_answers))
        accuracy = float(sum(correct_answers)) / float(len(correct_answers))
        return accuracy, sum(correct_answers)
    #print ("Predictions ",y_train,y_train_predicted.T)
    train_score, correct_answers_train = get_accuracy(y_train, y_train_predicted[0].T)
    test_score, correct_answers_test = get_accuracy(y_test, y_test_predicted[0])
    print("Train Done")

    return train_score, test_score, correct_answers_train, correct_answers_test

def test_sensitive_attr_constraint_cov(model, x_arr, y_arr_dist_boundary, x_control, thresh, verbose):

    
    """
    The covariance is computed b/w the sensitive attr val and the distance from the boundary
    If the model is None, we assume that the y_arr_dist_boundary contains the distace from the decision boundary
    If the model is not None, we just compute a dot product or model and x_arr
    for the case of SVM, we pass the distace from bounday becase the intercept in internalized for the class
    and we have compute the distance using the project function
    this function will return -1 if the constraint specified by thresh parameter is not satifsified
    otherwise it will reutrn +1
    if the return value is >=0, then the constraint is satisfied
    """

    assert(x_arr.shape[0] == x_control.shape[0])
    if len(x_control.shape) > 1: # make sure we just have one column in the array
        assert(x_control.shape[1] == 1)
    
    arr = []
    if model is None:
        arr = y_arr_dist_boundary # simply the output labels
    else:
        arr = np.dot(model, x_arr.T) # the product with the weight vector -- the sign of this is the output label
    
    arr = np.array(arr, dtype=np.float64)


    cov = np.dot(x_control - np.mean(x_control), arr ) / float(len(x_control))

        
    ans = thresh - abs(cov) # will be <0 if the covariance is greater than thresh -- that is, the condition is not satisfied
    # ans = thresh - cov # will be <0 if the covariance is greater than thresh -- that is, the condition is not satisfied
    if verbose is True:
        print ("Covariance is", cov)
        print ("Diff is:", ans)
        print
    return ans

def print_covariance_sensitive_attrs(model, x_arr, y_arr_dist_boundary, x_control, sensitive_attrs):


    """
    reutrns the covariance between sensitive features and distance from decision boundary
    """

    arr = []
    if model is None:
        arr = y_arr_dist_boundary # simplt the output labels
    else:
        arr = np.dot(model, x_arr.T) # the product with the weight vector -- the sign of this is the output label
    

    sensitive_attrs_to_cov_original = {}
    for attr in sensitive_attrs:

        attr_arr = x_control[attr]


        bin_attr = check_binary(attr_arr) # check if the attribute is binary (0/1), or has more than 2 vals
        if bin_attr == False: # if its a non-binary sensitive feature, then perform one-hot-encoding
            attr_arr = [int(x) for x in attr_arr]
            attr_arr_transformed, index_dict = get_one_hot_encoding(attr_arr)

        thresh = 0

        if bin_attr:
            cov = thresh - test_sensitive_attr_constraint_cov(None, x_arr, arr, np.array(attr_arr), thresh, False)
            sensitive_attrs_to_cov_original[attr] = cov
        else: # sensitive feature has more than 2 categorical values            
            
            cov_arr = []
            sensitive_attrs_to_cov_original[attr] = {}
            for attr_val, ind in index_dict.items():
                t = attr_arr_transformed[:,ind]
                cov = thresh - test_sensitive_attr_constraint_cov(None, x_arr, arr, t, thresh, False)
                sensitive_attrs_to_cov_original[attr][attr_val] = cov
                cov_arr.append(abs(cov))

            cov = max(cov_arr)
            
    return sensitive_attrs_to_cov_original


def get_correlations(model, x_test, y_predicted, x_control_test, sensitive_attrs):
    

    """
    returns the fraction in positive class for sensitive feature values
    """

    if model is not None:
        y_predicted = np.sign(np.dot(x_test, model))
        
    y_predicted = np.array(y_predicted)
    
    out_dict = {}
    for attr in sensitive_attrs:

        attr_val = []
        for v in x_control_test[attr]: attr_val.append(v)
        assert(len(attr_val) == len(y_predicted))


        total_per_val = defaultdict(int)
        attr_to_class_labels_dict = defaultdict(lambda: defaultdict(int))

        for i in range(0, len(y_predicted)):
            val = attr_val[i]
            label = y_predicted[i]

            if isinstance(label, np.float64):
                label = np.array([label])
            # val = attr_val_int_mapping_dict_reversed[val] # change values from intgers to actual names
            total_per_val[val] += 1
            attr_to_class_labels_dict[val][label[0]] += 1

        class_labels = y_predicted.tolist()

        local_dict_1 = {}
        for k1,v1 in attr_to_class_labels_dict.items():
            total_this_val = total_per_val[k1]

            local_dict_2 = {}
            for k2 in class_labels: # the order should be the same for printing
                if type(k2) is list:
                    v2 = v1[k2[0]]
                else:
                    v2 = v1[k2]

                f = float(v2) * 100.0 / float(total_this_val)

                if type(k2) is list:
                    local_dict_2[k2[0]] = f
                else:
                    local_dict_2[k2] = f
            local_dict_1[k1] = local_dict_2
        out_dict[attr] = local_dict_1

    return out_dict

def check_binary(arr):
    "give an array of values, see if the values are only 0 and 1"
    s = sorted(set(arr))
    if s[0] == 0 and s[1] == 1:
        return True
    else:
        return False

def get_avg_correlation_dict(correlation_dict_arr):
    # make the structure for the correlation dict

    correlation_dict_avg = {}

    for k,v in correlation_dict_arr.items():
        correlation_dict_avg[k] = {}
        for feature_val, feature_dict in v.items():
            correlation_dict_avg[k][feature_val] = {}

            for class_label, frac_class in feature_dict.items():
                correlation_dict_avg[k][feature_val][class_label] = []

    # populate the correlation dict
    if type(correlation_dict_arr) is not list:
        correlation_dict_arr = [correlation_dict_arr]
    for correlation_dict in correlation_dict_arr:

        for k,v in correlation_dict.items():
            for feature_val, feature_dict in v.items():

                for class_label, frac_class in feature_dict.items():
                    correlation_dict_avg[k][feature_val][class_label].append(frac_class)

    # now take the averages
    for k,v in correlation_dict_avg.items():
        for feature_val, feature_dict in v.items():
            for class_label, frac_class_arr in feature_dict.items():
                correlation_dict_avg[k][feature_val][class_label] = np.mean(frac_class_arr)

    return correlation_dict_avg

In [4]:
# Importing Numpy for scientific calculation
import numpy as np
# from helper import *
from copy import deepcopy
from scipy.optimize import minimize
class SVM:

    def predict(self,X,weights,lambd):
        return np.dot(X,weights) + np.sum(lambd*(weights**2))

    def findCost(self,Y,pred):
        value=Y*pred
        if value >1:
            cost=0
        else :
            cost=1-value
        return cost
    
    def training(self,x,y,x_control,loss_function,C,max_iter,lamb,epochs=500,lr=1, apply_fairness_constraints = 0, sensitive_attrs = ['sex'], sensitive_attrs_to_cov_thresh = {},gamma=None):
        '''
        This function return the model weight after training
        '''
    
        max_iter = max_iter # maximum number of iterations for the minimization algorithm

        if apply_fairness_constraints == 0:
            w = self.traindef(x,y,lamb,C,epochs,lr)
            return w
        
        elif apply_fairness_constraints == 1:
            print ("running Custom model")

            if gamma is not None and gamma !=0:

                w = minimize(fun=loss_function,
                             x0=np.random.rand(x.shape[1], ),
                             args=(x, y,C),
                             method='SLSQP',
                             options={"maxiter": max_iter},
                             constraints=[]
                             )

                old_w = deepcopy(w.x)

                def constraint_gamma_all(w, x, y, C,initial_loss_arr):

                    new_loss = loss_function(w, x, y,C)
                    old_loss = initial_loss_arr
                    return ((1.0 + gamma) * old_loss) - new_loss

                def constraint_protected_people(w, x,y):  # dont confuse the protected here with the sensitive feature protected/non-protected values -- protected here means that these points should not be misclassified to negative class
                    return np.dot(w, x.T)  # if this is positive, the constraint is satisfied

                def constraint_unprotected_people(w, old_loss, x, y,C):

                    new_loss = loss_function(w, np.array([x]), np.array(y),C)
                    return ((1.0 + gamma) * old_loss) - new_loss

                constraints = []
                unconstrained_loss_arr = loss_function(w.x, x, y,C)
                predicted_labels = np.sign(np.dot(w.x, x.T))
                for i in range(0, len(predicted_labels)):
                    if predicted_labels[i] == 1.0 and x_control[sensitive_attrs[0]][i] == 1.0:  # for now we are assuming just one sensitive attr for reverse constraint, later, extend the code to take into account multiple sensitive attrs
                        c = ({'type': 'ineq', 'fun': constraint_protected_people, 'args': (x[i], y[i])})  # this constraint makes sure that these people stay in the positive class even in the modified classifier
                        constraints.append(c)
                    else:
                        c = ({'type': 'ineq', 'fun': constraint_unprotected_people,
                              'args': (unconstrained_loss_arr, x[i], y[i],C)})
                        constraints.append(c)

                def cross_cov_abs_optm_func(weight_vec, x_in, x_control_in_arr):
                    cross_cov = (x_control_in_arr - np.mean(x_control_in_arr)) * np.dot(weight_vec, x_in.T)
                    return float(abs(sum(cross_cov))) / float(x_in.shape[0])

                w = minimize(fun=cross_cov_abs_optm_func,
                             x0=old_w,
                             args=(x, x_control[sensitive_attrs[0]]),
                             method='SLSQP',
                             options={"maxiter": 100000},
                             constraints=constraints
                             )

            else:

                constraints = self.get_constraint_list_cov(x, y,x_control,sensitive_attrs, sensitive_attrs_to_cov_thresh)
                f_args = (x, y,C)
                w = minimize(fun=loss_function,
                             x0=np.random.rand(x.shape[1], ),
                             args=f_args,
                             method='SLSQP',
                             options={"maxiter": max_iter},
                             constraints=constraints
                             )
        return w.x
        
    def predict(self,x_test,w):
        return np.sign(np.dot(np.array(x_test),w))
        
    
    def get_constraint_list_cov(self, x_train, y_train, x_control_train,sensitive_attrs, sensitive_attrs_to_cov_thresh):

        """
        get the list of constraints to be fed to the minimizer
        """

        constraints = []
        

        for attr in sensitive_attrs:


            attr_arr = x_control_train[attr]
            attr_arr = [int(x) for x in attr_arr]
            attr_arr_transformed, index_dict = get_one_hot_encoding(attr_arr)
                
            if index_dict is None: # binary attribute
                thresh = sensitive_attrs_to_cov_thresh[attr]

                c = ({'type': 'ineq', 'fun': test_sensitive_attr_constraint_cov, 'args':(x_train, y_train, attr_arr_transformed,thresh, False)})
                constraints.append(c)
            else: # otherwise, its a categorical attribute, so we need to set the cov thresh for each value separately


                for attr_val, ind in index_dict.items():
                    attr_name = attr_val                
                    thresh = sensitive_attrs_to_cov_thresh[attr][attr_name]
                
                    t = attr_arr_transformed[:,ind]
                    c = ({'type': 'ineq', 'fun': test_sensitive_attr_constraint_cov, 'args':(x_train, y_train,t ,thresh, False)})
                    constraints.append(c)



        return constraints
        
        
    def traindef(self,X,Y,lamb,C,epochs=500,lr=1):    # Input dimensions  X:(m,n) Y:(m,1)
        '''
        Training SVM using Gradient decent approach
        Parameters
        ----------
        X - Input features
        Y - Labels
        epochs - Number of epochs
        lr - learning rate for gradient decent
        Returns - The model weights
        '''

        print ("Train Default model")

        n,m=X.shape[1] ,X.shape[0]
        w= np.zeros((n, 1))

        for epoch in range(1,int(epochs)):
            for i,x in enumerate(X):
                x_train=x.reshape(1,-1)
                y_train = Y[i].reshape(1,1)
                y_hat= np.dot(x_train,w) +np.sum(lamb * (w ** 2))
                val=y_train *y_hat
                loss = [0  if val> 1 else 1 -val]
                if loss==0:
                    grad=np.zeros(w.shape)
                    w = w - lr* (grad + 2*lamb*w*C)
                else:
                    grad=(-y_train*x_train).T
                    w = w - lr*(grad + 2 *lamb*w*C )

        return w

In [5]:
import pandas as pd
import numpy as np
from sklearn import svm
import math

In [6]:
pd.set_option('display.max_rows', None)

In [7]:
##global variables
tz=7
tpr_by_fpr_min_thresh=0

tc=400
target_column_name='income'

In [8]:
data=pd.read_csv("datasets/adult.csv")
# data=data[:1000]

In [9]:
data['income'].replace(['<=50K', '>50K'],
                        [0, 1], inplace=True)

In [10]:
list_of_catagorical_vars=['workclass','education',
       'marital.status', 'occupation', 'relationship', 'race', 'sex', 'native.country',
]


In [11]:
from sklearn.preprocessing import LabelEncoder
encoded_to_orig_dict={}
for col_name in list_of_catagorical_vars:
    le = LabelEncoder()
    le.fit(data[col_name])
    data[col_name]=le.transform(data[col_name])
    

    encoded_to_orig_dict[col_name]= dict((v,k) for k,v in dict(zip(le.classes_, le.transform(le.classes_))).items())

In [12]:
encoded_to_orig_dict

{'workclass': {0: '?',
  1: 'Federal-gov',
  2: 'Local-gov',
  3: 'Never-worked',
  4: 'Private',
  5: 'Self-emp-inc',
  6: 'Self-emp-not-inc',
  7: 'State-gov',
  8: 'Without-pay'},
 'education': {0: '10th',
  1: '11th',
  2: '12th',
  3: '1st-4th',
  4: '5th-6th',
  5: '7th-8th',
  6: '9th',
  7: 'Assoc-acdm',
  8: 'Assoc-voc',
  9: 'Bachelors',
  10: 'Doctorate',
  11: 'HS-grad',
  12: 'Masters',
  13: 'Preschool',
  14: 'Prof-school',
  15: 'Some-college'},
 'marital.status': {0: 'Divorced',
  1: 'Married-AF-spouse',
  2: 'Married-civ-spouse',
  3: 'Married-spouse-absent',
  4: 'Never-married',
  5: 'Separated',
  6: 'Widowed'},
 'occupation': {0: '?',
  1: 'Adm-clerical',
  2: 'Armed-Forces',
  3: 'Craft-repair',
  4: 'Exec-managerial',
  5: 'Farming-fishing',
  6: 'Handlers-cleaners',
  7: 'Machine-op-inspct',
  8: 'Other-service',
  9: 'Priv-house-serv',
  10: 'Prof-specialty',
  11: 'Protective-serv',
  12: 'Sales',
  13: 'Tech-support',
  14: 'Transport-moving'},
 'relationshi